# Integrated Project

## Introduction

We are looking at data from Ice, an online retail store for video games. The data set includes categories based on genre, platform, rating, sales, release year, critic, and user scores.  

## Purpose

The purpose of this project is to collect historical data, and apply insights to determine whether a game will succeed or not in the future. These insights will be used to optimize capital allocation in regards to advertising potentially big winners. We will be determining total sales,  and the distribution  of sales across the different platforms. We will illustrate which platforms lead, and lag in sales. Data analysis will determine the most popular platforms and genres. Investigations will determine if the average user ratings of certain platforms are the same, or if the average ratings of certain genres are the same. 

## Reading and Viewing Data

In [ ]:
!pip install --user -U plotly_express

In [ ]:
# Import useful packages
import pandas as pd
import numpy as np
import math as mt
from scipy import stats as st
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# Read the dataframe
df = pd.read_csv('datasets/games.csv')

In [ ]:
# Visual of the Dataframe
display(df.head())

In [ ]:
# We see the columns need to be changed to lowercase
df.columns

In [ ]:
# Converting columns to lowercase
df.columns = df.columns.str.lower()

In [ ]:
# Info of dataframe
df.info()

We see some missing values in some columns, especially with the scores and rating. Sales data is all there, and all but 2 game names are present. 

In [ ]:
# Description of the dataset
df.describe()

In [ ]:
# Unique values of each column
df.nunique()

We have 31 different platforms spanning a time frame of 37 years. The dataset has 12 different genres, a range of 96 values for user score, and 8 different values for rating. Most games were sold in the North American region, while less games are sold in the Japanese region. The data shows the sales of 4 regions, which we can use to make summary data on total and average sales. There appears to be a large quantity of score and rating data missing. 

## Cleaning Data

Changing Float Columns to Integers

In [ ]:
# Visual of different user score values
df['user_score'].unique()

In [ ]:
# replacing TBD with a value
df['user_score'] = df['user_score'].replace(to_replace='tbd', value=0)

In [ ]:
# Need to convert user score from object to integer
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

In [ ]:
# Change release year to integer, years dont have decimals
df['year_of_release'] = df['year_of_release'].astype('int', errors='ignore')

### Checking for Duplicates

In [ ]:
# Checking for duplicates 
df.duplicated().sum()

In [ ]:
# Checking for duplicates filtering for same name and platform
df[df[['name', 'platform']].duplicated()].value_counts(ascending=False)

In [ ]:
# checking for duplicates based on same name, platform, genre
df[df[['name','platform', 'genre']].duplicated()]

In [ ]:
# looking for games with duplicate platform and year of release as hidden duplicates
df[df[['name','platform', 'year_of_release']].duplicated()]

In [ ]:
# Checking for Sonic
df[df['name']== 'Sonic the Hedgehog']

In [ ]:
# Looking for Need for Speed
df[df['name']== 'Need for Speed: Most Wanted']

In [ ]:
# Looking at Madden hidden duplicates
df[df['name']== 'Madden NFL 13']

In [ ]:
# Dropping the hidden duplicate
df = df.drop_duplicates(subset=['name','platform', 'year_of_release'], keep='first')

In [ ]:
# QC check
df[df[['name','platform', 'genre', 'year_of_release']].duplicated()]

In [ ]:
# looking for hidden duplicates
df.groupby(['name', 'platform'])['name'].value_counts(ascending=False)

We see Madden 13 has a hidden duplicate among the PS3 platform. We decide to keep the first, and drop the second instance, as the second has minimal data. Sonic and Need for speed do not have duplicate games, released on the same platform, in the same year. 

### Checking for Missing Values

In [ ]:
# Looking for missing values
df.isna().sum()

#### Missing Names

In [ ]:
# Visual of the rows of missing names
df[df['name'].isna()]

In [ ]:
# Dropping the rows of missing names
df = df.dropna(subset=['name', 'genre'])

#### Missing Year of Release

In [ ]:
# Looking for vales with missing release years
df[df['year_of_release'].isna()]

In [ ]:
# Mean release year per platform
df.groupby('platform')['year_of_release'].mean().round()

In [ ]:
# Median release year per platform
df.groupby('platform')['year_of_release'].median()

In [ ]:
# Percent of data with missing year
(df['year_of_release'].isna().sum() / df['year_of_release'].count()) * 100

In [ ]:
# fill year of release based on median of platform grouping
df = df.dropna(subset=['year_of_release'])

In [ ]:
# checking removal of missing values
df['year_of_release'].isna().sum()

In [ ]:
# Change release year to integer
df['year_of_release'] = df['year_of_release'].astype('int')

In [ ]:
# info on columns
df.info()

#### Missing Rating

In [ ]:
# Looking for missing ratings
df['rating'].isna().sum()

In [ ]:
# trying to figure out missing ratings
df.groupby(['name','genre', 'platform'])['rating'].value_counts()

In [ ]:
# The ratings of the titles, based on the platform
df.groupby(['name', 'platform'])['rating'].value_counts()

In [ ]:
# Grouping titles based on rating
df.groupby('name')['rating'].value_counts()

In [ ]:
# Rating count per title
df[['name', 'rating']].value_counts()

In [ ]:
# counting the different ratings of platforms
platform_rating = df[['platform', 'rating']].value_counts().reset_index()
display(platform_rating)

In [ ]:
# sorting by platform
platform_rating.sort_values('platform')

In [ ]:
# Best option 
genre_rating = df[['genre', 'rating']].value_counts()

We dropped the two entries with missing names, as they also had other critical data that was missing. We fill in missing year of release based on the median of the year the platform was released. This should have a minimal effect on our data, as we are filling a few hundred missing values. Then, we changed the year of release into an integer type, as we should not have decimals in our years. 

We fill TBD user scores with zero, to differentiate them from the other scores. Using a median value would change the results of the data. We believe these values are labeled as TBD, as a filler because data was not collected or missing. Another possibility is the users are still providing data on the scores of the games, or not enough data has been collected.  

We looked at grouping the data to determine a median value to replace the missing score and ratings data. The best option was to determine the ratings by genre. However, we decided to keep missing user score, critic score, and rating values as is. We decide to keep those missing values, as filling them in with mean or median values would alter the data, as roughly half of those data points are currently missing. We notice that those rows missing scores are also missing ratings. Therefore, the negative effect of these games on the data is limited. Also, we refrain from dropping those rows, as they still have crucial sales, genre, name, and platform data. 

## Feature Engineering

In [ ]:
# Changing game names to lowercase, platforms to uppercase
df['name'] = df['name'].str.lower()
df['platform'] = df['platform'].str.upper()

In [ ]:
# scaling down critic score
df['critic_score'] = df['critic_score'] / 10

In [ ]:
# Creating average score column
df['average_score'] = df[['critic_score', 'user_score']].mean(axis=1)

In [ ]:
# Adding total sales column
df['total_sales'] = df[['na_sales','eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

In [ ]:
# Adding average sales column
df['average_sales'] = df[['na_sales','eu_sales', 'jp_sales', 'other_sales']].mean(axis=1)

We scale down critic score to the same scale as user score, so we can get an average score column. Average score is the mean of user and critic scores. We add a total sales and an average sales column to the data. 

## Analyzing Data

### Game Releases

In [ ]:
# Number of games released in each year
game_years = df['year_of_release'].value_counts()
game_years = pd.DataFrame(game_years)

In [ ]:
# The years with the largest number of games released
game_years.head()

In [ ]:
# plot of games released each year
px.bar(game_years, title='Number of Games Released Each Year').show()

In [ ]:
# games released per genre
px.histogram(df, x='year_of_release', color='genre', title='Number of Games Released Per Genre').show()

In [ ]:
# games released per platform
px.histogram(df, x='year_of_release', color='platform', title='Number of Games Released Per Platform').show()

#### Conclusions

We see a steady increase in the number of games released from the early 90's. Game releases peak in 2008, then sharply decrease going into 2016. Sports and action games are the genres that have the most releases throughout the time period. The data also shows the changing of the various gaming platforms over time, as new platforms replace old generations. 

In [ ]:
# Looking at all the different platforms 
df['platform'].unique()

In [ ]:
# Looking at all the all the genres
df['genre'].unique()

In [ ]:
# Median critic score of each platform
df.groupby('platform')['critic_score'].median()

In [ ]:
# Counts of different user scores
df['user_score'].value_counts()

In [ ]:
# Median of user score per platform
df.groupby('platform')['user_score'].median()

In [ ]:
# Total sales based on median user score
df.groupby('total_sales')['user_score'].median()

In [ ]:
# Count of the different ratings 
df['rating'].value_counts()

In [ ]:
# Platform average and total sales
top_platforms = df.pivot_table(index='platform', values=['total_sales', 'average_sales'], aggfunc='sum').reset_index()

### Top Grossing Years

In [ ]:
# top grossing year
top_five_years = df.pivot_table(index='year_of_release', values=['total_sales', 'average_sales'], aggfunc='sum').reset_index()
top_five_years = top_five_years.nlargest(5, columns='total_sales')

In [ ]:
# top 5 grossing years
px.bar(top_five_years, x='year_of_release', y=['total_sales', 'average_sales'], barmode='group', title='Total Sales Per Top Release Year').show()

When it comes to sales, the top grossing years are from 2006 to 2010. The year 2009 had the most game sales in the entire 37 year period from 1985 to 2016. This represents the era of the PS3, X360, and the WII, which explains why these platforms were so popular. 

### Total and Average Sales Per Platform

In [ ]:
# platform total and average sales
px.bar(top_platforms, x='platform', y=['total_sales', 'average_sales'], barmode='group', title='Platform Sales').show()

We see that the PS2 was the most successful platform from 1985 to 2016. Other popular platforms are the X360, PS3, WII and DS. We also see that the upgraded version of these platforms are not yet as popular. This may be due to their competition with the predecessors, as the new versions are released about half a decade int the predecessors run. Looking to the future, one would expect the PS4 to dominate the other platforms, while the XONE, WIIU and 3DS would follow as the new generation platforms.

We will analyze the historic data for context, then analyze the most recent time periods to make predictions for 2017.



### Platform Sales by Region

In [ ]:
# Platform sales by region, average and total
platform_sales = df.pivot_table(index='platform', values=['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales', 'average_sales'], aggfunc='sum').reset_index()


In [ ]:
# platform regional sales
px.bar(platform_sales, x='platform', y=['eu_sales','na_sales', 
'jp_sales', 'other_sales'], width=1400, title='Regional Sales Per Platform').show()

Platform sales by region breaks down the sales based on the variable regions in our dataset. We see that North America leads in the sales of most of the platforms, followed by Europe. Japan regional sales shows which platforms users prefer there. 

### Top Grossing Platforms

In [ ]:
# top grossing platforms
top_five_platforms = df.pivot_table(index='platform', values=['eu_sales','na_sales', 
'jp_sales', 'other_sales', 'total_sales', 'average_sales'], aggfunc='sum').reset_index()
top_five_platforms = top_five_platforms.nlargest(5, columns='total_sales')


In [ ]:
# platform sales
px.bar(top_five_platforms, x='platform', y='total_sales',
 color='platform', title='Total Sales Per Platform').show()

In [ ]:
# platform total and average sales
px.bar(top_five_platforms, x='platform', y=['total_sales', 'average_sales'],
barmode='group', title='Sales Per Platform', width=1200).show()

In [ ]:
# platform sales boxplot
px.box(platform_sales, x='platform', y=['na_sales', 'eu_sales', 'jp_sales', 'other_sales'], title='Platform Sales').show()

As we saw from previous results, the top grossing platforms were the PS2, X360, PS3, WII and the DS. Note that these platforms are not the latest platforms to come to market, so they have had a longer lifespan to build sales. Surprisingly, the PS2 is still the leader, despite the PS3 holding the number 3 position, and the PS4 being released. The box plots illustrate the differences in the statistics of the platform sales. We see some platforms have a wide range, while others have an extremely tight range.  

### Top Grossing Genres

In [ ]:
# top grossing genre
top_five_genres = df.pivot_table(index='genre', values=['total_sales', 'average_sales'], aggfunc='sum').reset_index()
top_five_genres = top_five_genres.nlargest(5, columns='total_sales')


In [ ]:
# sales of top 5 genres
px.bar(top_five_genres, x='genre', y='total_sales',
 color='genre', width=1300, title='Total Sales Among Top Genres').show()

In [ ]:
# total and averag sales, top 5 genres
px.bar(top_five_genres, x='genre', y=['total_sales', 'average_sales'],
width=1300, barmode='group', title='Sales Among Top Genres').show()

In [ ]:
# share of top 5 genres
px.pie(top_five_genres, values='total_sales', names='genre').show()

The top grossing genres were action, sports, shooter, role playing, and platform. Now of the top 5 platforms, action games take a 29.6% market share, while sports takes up 22.6%. 

### Top Grossing Games

In [ ]:
# top grossing games
top_five_games = df.pivot_table(index='name', values=['total_sales', 'average_sales'], aggfunc='sum').reset_index()
top_five_games = top_five_games.nlargest(5, columns='total_sales')

In [ ]:
# sales of top 5 games
px.bar(top_five_games, x='name', y='total_sales', 
color='name', title='Sales Among Top Games').show()

In [ ]:
# total and average sales, top 5 games
px.bar(top_five_games, x='name', y=['total_sales', 'average_sales'], barmode='group', title='Sales Among Top Games').show()

The top grossing games of this time frame are WII sports, GTA V, Super Mario Bros, Tetris, and Mario Kart WII. These entries explain why the WII is one of the top 5 platforms by sales. This is also significant, as WII games are not played across platforms. WII sports being a sports genre game supports the conclusion of sports being one of the top selling genres. Since GTA V is an action game, this supports that genre being in the top five as well. 

### Sales Based on Critic Scores

In [ ]:
# Sales based on critic scores
critic_sales = df.pivot_table(index='critic_score', values=['eu_sales','na_sales', 
'jp_sales', 'other_sales', 'total_sales', 'average_sales'], aggfunc='sum').reset_index()

In [ ]:
# top grossing critic score
top_five_critic = critic_sales.nlargest(5, columns=['total_sales', 'average_sales'])

In [ ]:
# regional sales, critic scores
px.bar(critic_sales, x='critic_score', y=['eu_sales','na_sales', 
'jp_sales', 'other_sales'], title='Regional Sales Based on Critic Scores').show()

In [ ]:
# average sales, critic scores
px.bar(critic_sales, x='critic_score', y='average_sales', title='Average Sales Based on Critic Score').show()

In [ ]:
# total and average sales, critic scores
px.bar(top_five_critic, x='critic_score', y=['total_sales', 'average_sales'], barmode='group',
    title='Total and Average Sales Based on Critic Score').show()

In [ ]:
# correlation of sales and critic scores
df[['critic_score', 'eu_sales', 'jp_sales', 'na_sales', 'total_sales']].corr()

We see the bar graph of total sales skewed to the left, and distributed around a critic score of 8. The data suggest critic scores are a poor determinate of total sales. We see that the games with the highest scores, do not have the highest sales. The data shows us that games with a critic score of 8, have the highest total sales. Yet, games with lower critic scores do have lower sales.  Correlation data confirms a weak correlation between critic score and sales. 

### Sales Based on User Scores

In [ ]:
# Sorting by user score, then getting data on sales
user_sales = df.pivot_table(index='user_score', values=['eu_sales','na_sales', 
'jp_sales', 'other_sales', 'total_sales', 'average_sales'], aggfunc='sum').reset_index()


In [ ]:
#regional sales
fig = px.bar(user_sales, x='user_score', y=['eu_sales','na_sales', 
'jp_sales', 'other_sales'], title='Regional Sales Based on User Scores', )

fig.update_xaxes(range=[0.1, 10])
fig.show()

In [ ]:
# average sales
fig = px.bar(user_sales, x='user_score', y='average_sales', title='Average Sales Based on User Score')

fig.update_xaxes(range=[0.1, 10])
fig.show()

In [ ]:
# top grossing user score
top_five_user = user_sales.nlargest(5, columns=['total_sales', 'average_sales'])

In [ ]:
# sales of top scores
fig = px.bar(top_five_user, x='user_score', y=['total_sales', 'average_sales'], barmode='group', 
title='Total and Average Sales Based on User Score')

fig.update_xaxes(range=[6, 10])
fig.show()

In [ ]:
# correlation of user score and sales
df[['user_score', 'eu_sales', 'jp_sales', 'na_sales', 'total_sales']].corr()

We see the same trend in user scores and critic scores, when looking at sales data. We see a left skewed distribution around a score of 8. Then, higher rated games do not show higher total sales. Games with the user score of 8 show the highest total sales. Correlation confirms a weak correlation between user score and sales. 

### Sales Based on Average Scores

In [ ]:
# Sorting data by average score, then looking at sales
average_score_sales = df.pivot_table(index='average_score', values=['eu_sales','na_sales', 
'jp_sales', 'other_sales', 'total_sales', 'average_sales'], aggfunc='sum').reset_index()


In [ ]:
# regional sales and average scores
fig = px.bar(average_score_sales, x='average_score', y=['eu_sales','na_sales', 
'jp_sales', 'other_sales'], title='Regional Sales Based on Average Scores')

fig.update_xaxes(range=[0.1, 10])
fig.show()

In [ ]:
# average sales, average score
fig = px.bar(average_score_sales, x='average_score', y='average_sales', title='Average Sales Based on Average Score')

fig.update_xaxes(range=[0.1, 10])
fig.show()

Taking the average of user and critic scores slightly changes the top selling score to a 7.8. The shape and distribution of the sales stays mostly the same. 

### Yearly Sales

In [ ]:
# Dataframe of sales based on release year
yearly_sales = df.groupby('year_of_release')['total_sales', 'average_sales'].sum().sort_values(by='total_sales', ascending=False).reset_index()

In [ ]:
# total sales, year of release
px.bar(yearly_sales, x='year_of_release', y=['total_sales', 'average_sales'], title='Total Sales by Year of Release', barmode='group').show()

We see how sales are generally distributed from 2001 to 2013. The total sales rose from 1996 to its peak in 2008. Since 2008, we have seen a steady decline in sales, back to the levels of the late 80's. 

#### Yearly Sales Based on Region

In [ ]:
# yearly regional sales pivot table
yearly_region_sales = df.pivot_table(index='year_of_release', values=['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales', 'average_sales'], aggfunc='sum').reset_index()

In [ ]:
# plot pivot table
px.bar(yearly_region_sales, x='year_of_release', 
y=['na_sales', 'eu_sales', 'jp_sales', 'other_sales'], title='Regional Sales Based on Year of Release').show()

In [ ]:
# plot pivot table, average sales
px.bar(yearly_region_sales, x='year_of_release', 
y='average_sales', title='Average Sales Based on Year of Release').show()

Customers in North America tend to buy more games than customers of the other regions, generally throughout time. Sales in the Japanese region was most significant in the early 90's. The European region has seen a boost in sales starting from the mid 90's. 

#### Lifespan of the Platforms

In [ ]:
# lifespan of the different platforms
platform_lifespan = df.groupby('platform')['year_of_release'].value_counts()
platform_lifespan = pd.DataFrame(platform_lifespan)

In [ ]:
# reset the index for cleaner dataframe
platform_years = df.groupby(['platform', 'year_of_release'])['total_sales', 'average_sales'].count().reset_index(level=0)
platform_years = platform_years.reset_index()

In [ ]:
# Make a pivot table of platform and years of release, with total sales values
platform_pivot = platform_years.pivot_table(index='platform', columns='year_of_release', values='total_sales',)

In [ ]:
# filter pivot for recent years
relevant_pp = platform_pivot.loc[:, 2013:2016].dropna()

In [ ]:
# Adding a lifespan count column
relevant_pp['count'] = platform_pivot.loc[:, 2013:2016].dropna().sum(axis=1)

In [ ]:
# Extracting the count colum to show lifespan of the platforms, descending order
platform_count_years = relevant_pp.count(axis=1)
platform_count_years = platform_count_years.sort_values(ascending=False)
platform_count_years.reset_index()

In [ ]:
# statistical overview
platform_count_years.describe()

The platform with the longest lifespan, as of 2016, is the PC. This is intuitive, as the PC has been around for a long time, and so have PC games. This is an outlier, as most platforms now have been around for 5 years. When looking at data from 2013-2016, the median and mean lifespan is 5 years. Therefore, the measure of a successful platform is one that has sales for at least 5 years. Furthermore, we can generally expect a successful platform to survive more than 5 years. 

#### Data for every year is not significant for what we are trying to achieve. 

### Top Platforms from 2000-2016

In [ ]:
# Filtering platforms by choices
platform_pivot_filtered = platform_pivot.loc[['PS2', 'PS3', 'WII', 'X360', 'DS']].dropna(axis=1, how='all')
platform_pivot_filtered

In [ ]:
# filter out DS in 1985
final_platform_pivot =platform_pivot_filtered.loc[:,2000:2016]
final_platform_pivot = final_platform_pivot.reset_index()
final_platform_pivot

In [ ]:
# Total Sales column 
final_platform_pivot['total'] = final_platform_pivot.sum(axis=1)
final_platform_pivot

In [ ]:
# Look at the names of the columns
final_platform_pivot.columns

##### We filter out the DS value in 1985, as that was likely the first generation of the platform. The relevant version is the current one. 



In [ ]:
# list of years 
final_platform_pivot.columns.to_list()

#### Official Release Dates
        PS2 march, 2000

        PS3 november, 2006

        WII november, 2006

        X360 november, 2005

        DS november, 2004

We see that the pivot table of release years correctly illustrates the lifespan of the platforms. The PS2 was released in 2000, while the PS3 and WII were released in 2006. The XBOX 360 was released in 2005, while the new generation DS was released in 2004. The total lifespan of the PS2 was 12 years, but the platform was replaced by the PS3 in 2006, which led to steady declines in PS2 game sales. This could be attributed to people upgrading their platform to the PS3. The sales of the PS3 slowly increased as the PS2 sales decreased. It took 3 years for PS3 game sales to overtake PS2 game sales. We also see that the Xbox 360 came out in 2006, as the games sales of it and the PS3 would compete with each other. The lifespan of the PS3 seems to continue beyond 2016, while the Xbox 360 started leveling off in 2012. This is interesting, as Xbox game sales peaked first, and was not overtaken by PS3 game sales until 2011. The WII platform was released in 2005, and still sees a small number of sales, only beginning to level off in 2016. The Nintendo DS was released in 2004, and saw a lifespan of 10 years, leveling off in 2012 to no game sales in 2014. 

In [ ]:
# platform lifespans
px.bar(final_platform_pivot, x='platform', 
y=[ 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,], 
title='Platform Lifespans 2000 to 2016').show()

### Top Platforms of Period 2013-2016

In [ ]:
# Filter data by last 4 years
last_period_sales = yearly_sales.sort_values('year_of_release', ascending=False).head(4)

In [ ]:
# Filter parameters
key_platforms = ('PS2', 'X360', 'PS3', 'WII', 'DS')
key_years = [2013, 2014, 2015, 2016]

In [ ]:
# Filter dataframe based on platforms and years
df_filtered = df[df['platform'].isin(key_platforms) & df['year_of_release'].isin(key_years)]

In [ ]:
# total and average sales, recent years
px.bar(last_period_sales, x='year_of_release', y=['total_sales', 'average_sales'], barmode='group', 
    title='Total and Average Sales based on Year of Release').show()

### Sales of Last Four Years

Only data from the last period is relevant, as we are attempting to make a prediction for 2017. As such, older platforms and older years are not needed in the data for overall analysis. Here, we will use the last 4 years. 

## Most Relevant Platforms 2013-2016

In [ ]:
# Most relevant platforms from recent time period
platform_pivot.loc[['3DS', 'PC', 'PS4', 'PSV', 'XONE'], 2000:2016]

In [ ]:
# Filtering platforms from 2013 to 2016, by total sales in period
relevant_platforms = platform_pivot.loc[:,2013:2016].dropna(how='all')

These are the most relevant platforms, in the most relevant periods. These platforms represent a sample of the various platforms still being played. Furthermore, these are newer generation platforms that came out in the last couple of years, and have a few more years left in their life cycles. The oldest Platform is the PC. Platforms 3DS and PSV represent handheld options, while PS4 and XONE represent the period consoles for Sony and Microsoft, respectively. 

## Top 5 Most Relevant Platforms from 2013 to 2016

In [ ]:
# Filter for 5 platforms in the time period
most_relevant_platforms = relevant_platforms.loc[['3DS', 'PC', 'PS4', 'PSV', 'XONE']]
most_relevant_platforms

In [ ]:
# platform game releases, year
px.bar(most_relevant_platforms, title='Most Relevant Platforms from 2013 to 2016', width=1200).show()

In [ ]:
# total sales boxplot
px.box(most_relevant_platforms, title='Total Sales of Platforms from 2013 to 2016', width=1200).show()

The most profitable platform is the PS4, and the game sales are still increasing. PC sales are the lowest, yet PC as a platform has stood the test of time and still persists. The other platforms we need to pay attention to are PSV, XONE and 3DS. 

## Differences in Sales by Platform 2013 to 2016

In [ ]:
# Take the most relevant platform dataset
df2 = most_relevant_platforms[:]

In [ ]:
# Adding a column that totals platform sales
df2['period_total'] = df2.sum(axis=1)

In [ ]:
# List of most relevant platforms
most_relevant_platforms_list = ('3DS', 'PS4', 'PC', 'XONE', 'PSV')

In [ ]:
# Filter dataset by relevant platforms list, to get regional sales
df4 = platform_sales[platform_sales['platform'].isin(most_relevant_platforms_list)]

In [ ]:
# Sorted by EU sales
df4.sort_values(by='eu_sales')

In [ ]:
# Sorted by JP sales 
df4.sort_values(by='jp_sales')

In [ ]:
# Sorted by NA sales
df4.sort_values(by='na_sales')

In [ ]:
# Cum sum of sales in regions
df4_sum =df4.cumsum()
df4_sum = df4_sum[4:]
df4_sum

### 3DS Platform

In [ ]:
# DS ratios of total sales
ds = pd.DataFrame((df4.iloc[0,1:] / df4_sum.iloc[0, 1:]) * 100)
ds.columns= ['DS']
ds.reset_index()

In [ ]:
# DS ratio of total sales list
ds_sales = ds['DS'].iloc[1:5].to_list()
ds_sales

In [ ]:
# DS regional sales numbers
ds_box = pd.DataFrame(df4.iloc[0,1:])
ds_box.columns= ['3DS']
ds_box.reset_index()

In [ ]:
# DS regional sales numbers list
ds_box_sales = ds_box['3DS'].iloc[1:5].to_list()

### PC Platform

In [ ]:
# PC ratios of total sales
pc = pd.DataFrame((df4.iloc[1,1:] / df4_sum.iloc[0, 1:]) * 100)
pc.columns = ['PC']
pc.reset_index()

In [ ]:
# PC ratio of total sales list
pc_sales = pc['PC'].iloc[1:5].to_list()

In [ ]:
# PC regional sales numbers
pc_box = pd.DataFrame(df4.iloc[1,1:])
pc_box.columns= ['PC']
pc_box.reset_index()

In [ ]:
# PC regional sales numbers  list
pc_box_sales = pc_box['PC'].iloc[1:5].to_list()

### PS4 Platform

In [ ]:
# PS4 ratios of total sales
ps4 = pd.DataFrame((df4.iloc[2,1:] / df4_sum.iloc[0, 1:]) * 100)
ps4.columns = ['PS4']
ps4.reset_index()

In [ ]:
# PS4 ratios of total sales list
ps4_sales = ps4['PS4'].iloc[1:5].to_list()

In [ ]:
# PS4 regional sales numbers
ps4_box = pd.DataFrame(df4.iloc[2,1:])
ps4_box.columns = ['PS4']
ps4_box.reset_index()

In [ ]:
# PS4 regional sales numbers list
ps4_box_sales = ps4_box['PS4'].iloc[1:5].to_list()

### PSV Platform

In [ ]:
# PSV ratios of total sales
psv = pd.DataFrame((df4.iloc[3,1:] / df4_sum.iloc[0, 1:]) * 100)
psv.columns = ['PSV']
psv.reset_index()

In [ ]:
# PSV ratios of sales list
psv_sales = psv['PSV'].iloc[1:5].to_list()

In [ ]:
# PSV sales numbers
psv_box = pd.DataFrame(df4.iloc[3,1:])
psv_box.columns = ['PSV']
psv_box.reset_index()

In [ ]:
# PSV sales numbers list
psv_box_sales = psv_box['PSV'].iloc[1:5].to_list()

### XONE Platform

In [ ]:
# XONE ratios of total sales
xone = pd.DataFrame((df4.iloc[4,1:] / df4_sum.iloc[0, 1:]) * 100)
xone.columns = ['XONE']
xone.reset_index()

In [ ]:
# XONE ratios of sales list
xone_sales = xone['XONE'].iloc[1:5].to_list()

In [ ]:
# XONE regional sales
xone_box = pd.DataFrame(df4.iloc[4,1:])
xone_box.columns = ['XONE']
xone_box.reset_index()

In [ ]:
# XONE regional sales list
xone_box_sales = xone_box['XONE'].iloc[1:5].to_list()

### Overall Platform Results

In [ ]:
x_data = ['3DS', 'PC', 'PS4', 'PSV', 'XONE']

y_data = [ds_box_sales , pc_box_sales, ps4_box_sales, psv_box_sales, xone_box_sales]

colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
          'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)']

fig = go.Figure()

for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=2,
            line_width=1)
        )

fig.update_layout(
    title='Platform Sales from 2013 to 2016',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=10,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig.show()

### Conclusions

The box plots show the differences, and similarities in the total sales of each platform from 2013 to 2016. All platforms have a fairly broad spectrum, except for the PSV, which has a tight range. Statistical testing will determine if the mean sales of the regions are different from one another. 

## Market Share per Platform, by Region

In [ ]:
# DS data merged with PC
pt_1 = ds.T.reset_index().merge(pc.T.reset_index(), how='outer')

In [ ]:
# Merge again with PS4
pt_2 = pt_1.merge(ps4.T.reset_index(), how='outer')

In [ ]:
# Merge again with PSV
pt_3 = pt_2.merge(psv.T.reset_index(), how='outer')

In [ ]:
# Platform Market share percentages 
pt_final = pt_3.merge(xone.T.reset_index(), how='outer')
pt_final

### Euorope

In [ ]:
# platform market share, Europe
px.pie(pt_final, values=pt_final['eu_sales'], names=['3DS', 'PC', 'PS4', 'PSV', 'XONE'], title='European Region Platform Market Share').show()

### North America

In [ ]:
# platform market share, North America
px.pie(pt_final, values=pt_final['na_sales'], names=['3DS', 'PC', 'PS4', 'PSV', 'XONE'], title='North American Region Platform Market Share').show()

### Japan

In [ ]:
# platform market share, Japan
px.pie(pt_final, values=pt_final['jp_sales'], names=['3DS', 'PC', 'PS4', 'PSV', 'XONE'], title='Japanese Region Platform Market Share').show()

### Other

In [ ]:
# platform market share, Other
px.pie(pt_final, values=pt_final['other_sales'], names=['3DS', 'PC', 'PS4', 'PSV', 'XONE'], title='Other Region Platform Market Share').show()

## Conclusions

The market share of each relevant platform, depends on the sales region. In the European region, a vast majority of games are sold for PC and PS4 platforms. In the North American region, market share is more or less evenly split among PC, PS4, XONE and 3DS platforms. In the Japanese region, the 3DS takes up 72.4% market share among the top 5 platforms. The other region sees about a 46% market share for the PS4, and then a 21% market share for PC, among the top platforms. 

## Statistical Testing of Mean Sales

### Comparison of 3DS and PC

#### The mean sales of 3DS and PC are the same ?

    Null Hypothesis : The mean sales of 3DS and PC are the same

In [ ]:
# Comparison of 3DS and PC

#Null Hypothesis

## The mean sales of 3DS and PC are the same


alpha = 0.05

results = st.ttest_ind(ds_box_sales, pc_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of 3DS and PS4

#### The mean sales of 3DS and PS4 are the same ?

    Null Hypothesis : The mean sales of 3DS and PS4 are the same

In [ ]:
# Comparison of 3DS and PS4

#Null Hypothesis

## The mean sales of 3DS and PS4 are the same


alpha = 0.05

results = st.ttest_ind(ds_box_sales, ps4_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of 3DS and PSV

#### The mean sales of 3DS and PSV are the same ? 

    Null Hypothesis : The mean sales of 3DS and PSV are the same

In [ ]:
# Comparison of 3DS and PSV

#Null Hypothesis

## The mean sales of 3DS and PSV are the same


alpha = 0.05

results = st.ttest_ind(ds_box_sales, psv_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of 3DS and XONE

#### The mean sales of 3DS and XONE are the same ?

    Null Hypothesis : The mean sales of 3DS and XONE are the same

In [ ]:
# Comparison of 3DS and XONE

#Null Hypothesis

## The mean sales of 3DS and XONE are the same


alpha = 0.05

results = st.ttest_ind(ds_box_sales, xone_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of PC and PS4

#### The mean sales of PC and PS4 are the same ?

    Null Hypothesis : The mean sales of PC and PS4 are the same

In [ ]:
# Comparison of PC and PS4

#Null Hypothesis

## The mean sales of PC and PS4 are the same


alpha = 0.05

results = st.ttest_ind(pc_box_sales, ps4_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of PC and PSV

#### The mean sales of PC and PSV are the same ?

    Null Hypothesis : The mean sales of PC and PSV are the same

In [ ]:
# Comparison of PC and PSV

#Null Hypothesis

## The mean sales of PC and PSV are the same


alpha = 0.05

results = st.ttest_ind(pc_box_sales, psv_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of PC and XONE

#### The mean sales of PC and XONE are the same ?

    Null Hypothesis : The mean sales of PC and XONE are the same

In [ ]:
# Comparison of PC and XONE

#Null Hypothesis

## The mean sales of PC and XONE are the same


alpha = 0.05

results = st.ttest_ind(pc_box_sales, xone_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of PS4 and PSV

#### The mean sales of PS4 and PSV are the same ?

    Null Hypothesis : The mean sales of PS4 and PSV are the same

In [ ]:
# Comparison of PS4 and PSV

#Null Hypothesis

## The mean sales of PS4 and PSV are the same


alpha = 0.05

results = st.ttest_ind(ps4_box_sales, psv_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of PS4 and XONE

#### The mean sales of PS4 and XONE are the same ?

    Null Hypothesis : The mean sales of PS4 and XONE are the same

In [ ]:
# Comparison of PS4 and XONE

#Null Hypothesis

## The mean sales of PS4 and XONE are the same


alpha = 0.05

results = st.ttest_ind(ps4_box_sales, xone_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Comparison of PSV and XONE

#### The mean sales of PSV and XONE are the same ?

    Null Hypothesis : The mean sales of PSV and XONE are the same

In [ ]:
# Comparison of PSV and XONE

#Null Hypothesis

## The mean sales of PSV and XONE are the same


alpha = 0.05

results = st.ttest_ind(psv_box_sales, xone_box_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

## Conclusions

Using statistical t test of the mean sales of the different platforms, we do not see many differences. The only difference we can attest to with more than a 95% degree of certainty is between the 3DS and the PSV. We fail to reject the null hypotheses that the mean sales for the other platforms are different from each other. We used the standard alpha of 0.05, so that we were at least 95% sure of our results. We chose the null hypotheses to be the mean sales of the corresponding platforms were the same. The lack of difference seen could be attributed to the variance, and hence, the range of the sales. 

## Differences in Sales by Region

In [ ]:
# Filtering data frame for most relevant platforms and key years 2013-2016
df3 = df[df['platform'].isin(most_relevant_platforms_list) & df['year_of_release'].isin(key_years)]
df3.head()

In [ ]:
# Filter by EU region sales
df_eu = df4[['platform', 'eu_sales']]

In [ ]:
# Convert EU data to list
df_eu_sales = df_eu['eu_sales'].to_list()

In [ ]:
# Filter by JP region sales
df_jp = df4[['platform', 'jp_sales']]


In [ ]:
# Convert JP data to list
df_jp_sales = df_jp['jp_sales'].to_list()

In [ ]:
# Filter for NA region sales
df_na = df4[['platform', 'na_sales']]

In [ ]:
# Convert NA sales to list
df_na_sales = df_na['na_sales'].to_list()

In [ ]:
# Filter for Other region sales
df_other = df4[['platform', 'other_sales']]

In [ ]:
# Convert other region sales to list
df_other_sales = df_other['other_sales'].to_list()

### Overal Results

In [ ]:
x_data = ['Europe', 'Japan', 'North America', 'Other']

y_data = [df_eu_sales , df_jp_sales, df_na_sales, df_other_sales]

colors = ['rgba(44, 160, 101, 0.5)',
          'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']

fig = go.Figure()

for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=2,
            line_width=1)
        )

fig.update_layout(
    title='Regional Sales from 2013 to 2016',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=10,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig.show()

## Conclusions

We see the data for the regional sales from 2013 to 2016. It is difficult to predict differences, so we would need to conduct statistical testing of means. 

## Statistical Testing of Sales Means by Region

### Europe and Japan

    Null Hypothesis : The mean sales of EU and JP are the same

In [ ]:
# Comparison of EU and JP

#Null Hypothesis

## The mean sales of EU and JP are the same


alpha = 0.05

results = st.ttest_ind(df_eu_sales, df_jp_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Europe and North America

    Null Hypothesis : The mean sales of EU and NA are the same

In [ ]:
# Comparison of EU and NA

#Null Hypothesis

## The mean sales of EU and NA are the same


alpha = 0.05

results = st.ttest_ind(df_eu_sales, df_na_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Europe and Other

    Null Hypothesis : The mean sales of EU and Other are the same

In [ ]:
# Comparison of EU and Other

#Null Hypothesis

## The mean sales of EU and Other are the same


alpha = 0.05

results = st.ttest_ind(df_eu_sales, df_other_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Japan and North America

    Null Hypothesis : The mean sales of JP and NA are the same

In [ ]:
# Comparison of JP and NA

#Null Hypothesis

## The mean sales of JP and NA are the same


alpha = 0.05

results = st.ttest_ind(df_jp_sales, df_na_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### Japan and Other

    Null Hypothesis : The mean sales of JP and NA are the same

In [ ]:
# Comparison of JP and NA

#Null Hypothesis

## The mean sales of JP and NA are the same


alpha = 0.05

results = st.ttest_ind(df_jp_sales, df_na_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

### North America and Other

    Null Hypothesis : The mean sales of NA and Other are the same

In [ ]:
# Comparison of NA and Other

#Null Hypothesis

## The mean sales of NA and Other are the same


alpha = 0.05

results = st.ttest_ind(df_na_sales, df_other_sales)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

## Conclusions

We see that most of the mean regional sales are not different from each other. The one exception lies with the mean sales, of North American and Other. We chose our null hypotheses to be similarities in mean sales, and our alpha value to be 0.05. This alpha allows us to be at least 95% certain of our results. 

## User and Professional Reviews Effect on PS4 Sales

In [ ]:
# Creating a filter
ps4_list = ['PS4']

In [ ]:
# filtering data frame for PS4 
df_ps4 = df3[df3['platform'].isin(ps4_list)]

In [ ]:
# Sorting the resulting data set by scores and total sales
ps4_score_sales = df_ps4[['user_score', 'critic_score','average_score', 'total_sales']].sort_values(by='total_sales', ascending=False)

In [ ]:
# correlation between scores and total sales
ps4_score_sales.corr()

In [ ]:
# sales based on review scores
px.scatter(ps4_score_sales, x=['user_score', 'critic_score', 'average_score'], 
y='total_sales', trendline='ols', trendline_color_override='black',
title='PS4 Total Sales Based on Review Scores').show()

## Conclusions

We see a very weak correlation between reviews and sales, when considering user and critic reviews. Critic scores show a stronger relationship with sales, when compared to user scores, yet that relationship is fairly weak. Therefore, scores are not a strong predictor of total sales. 

## Which platforms are leading in sales from 2013 to 2016?

### Top Platform Sales

In [ ]:
# Filter by 2013 to 2016
df5 = df[df['year_of_release'].isin(key_years)]

In [ ]:
# group the platforms by sales
df6 = df5.groupby('platform')['eu_sales', 'jp_sales', 'na_sales', 'other_sales', 'average_sales', 'total_sales'].sum()

In [ ]:
# Sort platforms by highest total sales
df6.sort_values(by='total_sales',ascending=False).head(10)

The PS4 leads in sales for the relevant time period. Third, is the PS3, which appears to be shrinking as PS4 sales increase. This is because the PS4 is the updated version of the PS3, so they compete in sales. The sales trends suggest customers are deciding to upgrade their platform. The next platform on the list is the XONE, which is the updated version of the X360. We see similar trends with  playstation. Fourth on the total sales ranking ins the 3DS, and 6th is the WIIU. 

## Comparing Game Sales on Various Platforms

In [ ]:
# Top games dataset
top_five_games

Wii sports and mario kart are only on the Wii platform. Super smash bros and Tetris may not be on cross platforms either.

In [ ]:
# Top 20 Games by total sales from 2013-2016
df5.pivot_table(index='name', values=['total_sales', 'average_sales'], 
aggfunc='sum').reset_index().sort_values(by='total_sales', ascending=False).head(20)

Looking at Grand Theft Auto V, Call of Duty: Ghost, Minecraft, and FIFA 15, as they appear on a number of platforms. Nevertheless, many games found on Nintendo platforms are not present on the other console platforms. 

In [ ]:
# Pivot table of games and their platform, aggregated by total sales
df7 = df5.pivot_table(index=['name', 'platform'], values='total_sales', aggfunc='sum').reset_index()

In [ ]:
# Creating filters
cross_games = ('grand theft auto v', 'call of duty: ghosts', 'minecraft', 'fifa 15')
cross_platforms = ('PS4', 'XONE', 'WIIU', 'PC', '3DS')

In [ ]:
# Applying filter to our data set
df_cross = df7[df7['name'].isin(cross_games) & df7['platform'].isin(cross_platforms)]

In [ ]:
# Seeing which platforms have the most games in our selection
df7[df7['platform'].isin(cross_platforms)].sort_values(by=['name', 'platform']).value_counts(subset='platform')

In [ ]:
# Making a COD data set
cod = df_cross[df_cross['name']=='call of duty: ghosts']

In [ ]:
#making a GTA data set
gta = df_cross[df_cross['name']=='grand theft auto v']

In [ ]:
px.bar(cod, x='platform', y='total_sales', color='platform', title='COD: Ghost Platform Sales').show()

In [ ]:
# GTA V sales
px.bar(gta, x='platform', y='total_sales', color='platform', title='GTA V Platform Sales').show()

The data shows that very few games are sold on multiple platforms, and even less sold on WIIU and 3DS, with other platforms. This suggests the WIIU and 3DS ecosystems are mostly closed off to the other platforms, as 3DS has the second highest amount of games, yet we do not see those games on other platforms. We do see cross platform games among PC, PS4, and XONE consoles. Where games appear cross platform on PS4, PC, and XONE, we see PS4 games sales dominate. This is further demonstrated by the PS4 leading in the number of different games sold. Sales are then second most popular on the XONE platform. We see this trend with Call of Duty: Ghosts, and Grand Theft Auto V, two of the most popular platform games. 

## General Distribution of Games by Genre and Rating

### Genre

In [ ]:
# Sorting data by genre
genres = df5.groupby('genre')['total_sales'].sum().sort_values(ascending=False).reset_index()


In [ ]:
# sales by genre
px.bar(genres, x='genre', y='total_sales', color='genre', title='Total Sales by Genre 2013-216').show()

### Rating

In [ ]:
# creating a rating pivot table 
df_rating = df5.pivot_table(index=['name', 'platform', 'rating'], values='total_sales', aggfunc='sum').reset_index()

In [ ]:
# Indexing for the total sales based on rating
df_rating_sales = df_rating.groupby('rating')['total_sales'].sum().sort_values(ascending=False).reset_index()


In [ ]:
# game rating sales
px.bar(df_rating_sales, x='rating', y='total_sales', color='rating', title='Game Rating Sales 2013-2016').show()

### Conclusions

Action games are the most profitable genre, followed by shooter and sports games. The genres with lower sales may have a smaller audience, or are geared for a subset of customers, such as children. We can see when evaluating sales based on rating, mature games sell more. A logical assumption would be that action and shooter games are for mature audiences. 

## Top Platforms Per Region

In [ ]:
# Filtering dataframe for key years 2013 to 2016
df8 = df[df['year_of_release'].isin(key_years)]

In [ ]:
# grouping result by platform, then looking at sales data
df9 = df8.groupby('platform')[['total_sales','eu_sales', 'jp_sales', 'na_sales', 'other_sales']].sum().reset_index()


In [ ]:
# platfrom global market share
px.pie(df9, values='total_sales', names='platform', title='Global Market Share by Platform').show()

### European Region Platform Market Shares

In [ ]:
# European top platforms
df_top_eu = df9.sort_values(by='eu_sales', ascending=False).head()
df_top_eu = df_top_eu[['platform','eu_sales']].reset_index()

In [ ]:
# Cum sum of sales in europe
df_top_eu_sum =df_top_eu.cumsum()
df_top_eu_sum = df_top_eu_sum[4:]
df_top_eu_sum = df_top_eu_sum['eu_sales'].to_list()

In [ ]:
# multiply market share by 100
df_top_eu['market_share']= (df_top_eu['eu_sales'] / df_top_eu_sum) *100

In [ ]:
# european market share
px.pie(df_top_eu, values='market_share', names='platform', title='European Region Platform Market Shares').show()

### Japanese Region Platform Market Shares

In [ ]:
# top japanese platforms
df_top_jp = df9.sort_values(by='jp_sales', ascending=False).head()
df_top_jp = df_top_jp[['platform','jp_sales']].reset_index()


In [ ]:
# Cum sum of sales in japan
df_top_jp_sum =df_top_jp.cumsum()
df_top_jp_sum = df_top_jp_sum[4:]
df_top_jp_sum = df_top_jp_sum['jp_sales'].to_list()

In [ ]:
# market share percentage
df_top_jp['market_share']= (df_top_jp['jp_sales'] / df_top_jp_sum) *100

In [ ]:
# Japan platform market share
px.pie(df_top_jp, values='market_share', names='platform', title='Japanese Region Platform Market Shares').show()

### North American Region Platform Market Shares

In [ ]:
# Top north american platforms
df_top_na = df9.sort_values(by='na_sales', ascending=False).head()
df_top_na = df_top_na[['platform','na_sales']].reset_index()

In [ ]:
# Cum sum of sales in north america
df_top_na_sum =df_top_na.cumsum()
df_top_na_sum = df_top_na_sum[4:]
df_top_na_sum = df_top_na_sum['na_sales'].to_list()

In [ ]:
# market share percentage
df_top_na['market_share']= (df_top_na['na_sales'] / df_top_na_sum) *100

In [ ]:
# north america platform market share
px.pie(df_top_na, values='market_share', names='platform', title='North American Region Platform Market Shares').show()

### Other Region Platform Market Shares

In [ ]:
# other region top platform sales
df_top_other = df9.sort_values(by='other_sales', ascending=False).head()
df_top_other = df_top_other[['platform','other_sales']].reset_index()


In [ ]:
# Cum sum of sales in north america
df_top_other_sum =df_top_other.cumsum()
df_top_other_sum = df_top_other_sum[4:]
df_top_other_sum = df_top_other_sum['other_sales'].to_list()

In [ ]:
# market share percentage
df_top_other['market_share']= (df_top_other['other_sales'] / df_top_other_sum) *100

In [ ]:
# other region market share
px.pie(df_top_other, values='market_share', names='platform', title='Other Region Platform Market Shares').show()

### Conclusions

The platforms with the top market shares generally differ region to region. The PS4 has the largest market share in Europe, followed by the PS3, XONE, X360, and then the 3DS. Japan market share is dominated by the 3DS. The 3DS is followed by the PS3, PSV, PS4, and then the WIIU. The North American market leads with the PS4, and then close after is the XONE. after those two, market share is split among the X360, PS3, and 3DS in that order. Looking at the other region, PS4 game sales take a lion share of the market at 44.8%. Following that platform is the PS3, XONE, X360, and then the 3DS. Overall, the PS4 seems to predominate most markets out of the Japanese region, where it takes only 11.7% of the market. The PS3 and XONE are also fairly strong in their market shares, but only out of the Japanese region. Japan has different tastes when it comes to platforms. This region prefers the 3DS, and is also somewhat fond of the WIIU, which is a platform not seen in the top five of any other regions. 

## Top Genres Per Region

In [ ]:
# Categorizing datafarame by genre and sum of sales
df10 = df8.groupby('genre')[['total_sales','eu_sales', 'jp_sales', 'na_sales', 'other_sales']].sum().reset_index()

In [ ]:
# market share, genre
px.pie(df10, values='total_sales', names='genre', title='Global Market Share by Genre').show()

### European Region Top 5 Genres Market Share

In [ ]:
# Top 5 european region genres
df_genre_eu = df10.sort_values(by='eu_sales', ascending=False).head()
df_genre_eu = df_genre_eu[['genre','eu_sales']].reset_index()

In [ ]:
# Cum sum of sales in europe
df_genre_eu_sum =df_genre_eu.cumsum()
df_genre_eu_sum = df_genre_eu_sum[4:]
df_genre_eu_sum = df_genre_eu_sum['eu_sales'].to_list()

In [ ]:
# Creating market share column
df_genre_eu['market_share']= (df_genre_eu['eu_sales'] / df_genre_eu_sum) *100


In [ ]:
# European genre market share
px.pie(df_genre_eu, values='market_share', names='genre', title='European Region Genre Market Shares').show()

### Japanese Region Top 5 Genres Market Share

In [ ]:
# Top 5 japanese region genres
df_genre_jp = df10.sort_values(by='jp_sales', ascending=False).head()
df_genre_jp = df_genre_jp[['genre','jp_sales']].reset_index()

In [ ]:
# Cum sum of sales in japan
df_genre_jp_sum =df_genre_jp.cumsum()
df_genre_jp_sum = df_genre_jp_sum[4:]
df_genre_jp_sum = df_genre_jp_sum['jp_sales'].to_list()

In [ ]:
# Creating market share column
df_genre_jp['market_share']= (df_genre_jp['jp_sales'] / df_genre_jp_sum) *100

In [ ]:
# Japan genre market share
px.pie(df_genre_jp, values='market_share', names='genre', title='Japanese Region Genre Market Shares')

### North American Region Top 5 Genres Market Share

In [ ]:
# Top 5 north american region genres
df_genre_na = df10.sort_values(by='na_sales', ascending=False).head()
df_genre_na = df_genre_na[['genre','na_sales']].reset_index()

In [ ]:
# Cum sum of sales in japan
df_genre_na_sum =df_genre_na.cumsum()
df_genre_na_sum = df_genre_na_sum[4:]
df_genre_na_sum = df_genre_na_sum['na_sales'].to_list()

In [ ]:
# Creating market share column
df_genre_na['market_share']= (df_genre_na['na_sales'] / df_genre_na_sum) *100

In [ ]:
# North America genre market share
px.pie(df_genre_na, values='market_share', names='genre', title='North American Region Genre Market Shares').show()

### Other Region Top 5 Genres Market Share

In [ ]:
# Top 5 other region genres
df_genre_other = df10.sort_values(by='other_sales', ascending=False).head()
df_genre_other = df_genre_other[['genre','other_sales']].reset_index()

In [ ]:
# Cum sum of sales in japan
df_genre_other_sum =df_genre_other.cumsum()
df_genre_other_sum = df_genre_other_sum[4:]
df_genre_other_sum = df_genre_other_sum['other_sales'].to_list()

In [ ]:
# Creating market share column
df_genre_other['market_share']= (df_genre_other['other_sales'] / df_genre_other_sum) *100

In [ ]:
# Other genre market share
px.pie(df_genre_other, values='market_share', names='genre', title='Other Region Genre Market Shares').show()

### Conclusions

The European region prefers action games, followed by shooters and sports. Role playing and racing take a small portion of the market share. In the Japanese region, role playing and then action games take a majority of the market share. Miscellaneous, fighting, and shooter genres take a smaller portion of the Japanese market. In North America, action games predominate, followed by shooter. Then we have sports, role playing, and finally miscellaneous. In other region, we see action is first, shooter is second, and sports is third. The last two are role playing and miscellaneous. Overall, the main trend is action, shooter, sports, role playing, in all regions beside Japan. In Japan, they prefer role playing and action games. 

## ESRB Ratings Affect Regional Sales

In [ ]:
# creating pivot based on ratings, then collect sales
df_esrb = df5.pivot_table(index=['name', 'rating'], 
values=['eu_sales','jp_sales', 'na_sales', 'other_sales','total_sales'], 
aggfunc='sum').reset_index()

In [ ]:
# grouping by rating, taking sum of sales data
df_esrb_sales = df_esrb.groupby('rating')[['eu_sales','jp_sales', 'na_sales', 'other_sales']].sum().reset_index()

In [ ]:
# sales based on rating
px.bar(df_esrb_sales, x='rating', y=['eu_sales','jp_sales', 'na_sales', 'other_sales'], 
    barmode='group', title='Regional Sales Based on ESRB Rating').show()

### Conclusions

In the North American region, customers prefer games rated M, and then rated E. The same holds true for the European and other region. In the Japanese region, customers prefer T, then E, followed by M. It appears that these trends share a relationship with the genre preferences of the respective regions. It would stand to reason that shooter and action games are rated M, while sports games are rated E. Role playing games must therefore be rated as T or E. 

## Hypothesis Testing

### Average user rating of XONE and PC are the same

    Null Hypothesis : The mean user rating of XONE and PC are the same

In [ ]:
# list of platfroms to filter 
xone_list = ['XONE']
pc_list = ['PC']

In [ ]:
# Filtered dataframe
df_xone_pc = df[df['platform'].isin(xone_list) | df['platform'].isin(pc_list)].dropna()

Have to drop missing values for hypothesis testing to work

In [ ]:
# Filter again by XONE
xone_user_list = df_xone_pc[df_xone_pc['platform'].isin(xone_list)]

In [ ]:
# List of user scores
xone_user_list = xone_user_list['user_score'].to_list()

In [ ]:
# Filter again by PC
pc_user_list = df_xone_pc[df_xone_pc['platform'].isin(pc_list)]

In [ ]:
# Convert values to list
pc_user_list = pc_user_list['user_score'].to_list()

In [ ]:
# Comparison of XONE and PC user ratings

#Null Hypothesis

## The mean user ratings of XONE and PC are the same


alpha = 0.05

results = st.ttest_ind(xone_user_list, pc_user_list)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

##### The average user ratings of XONE and PC platforms are different.

### Average user rating of action and sports are the same

    Null Hypothesis : The mean user rating of action and sports are the same

In [ ]:
# Creating action and sports filter
action = ['Action']
sports = ['Sports']

In [ ]:
# Filtered dataframe
df_action_sports = df[df['genre'].isin(action) | df['genre'].isin(sports)].dropna()

In [ ]:
# Filter again by action
action_user_list = df_action_sports[df_action_sports['genre'].isin(action)]

In [ ]:
# List of user scores
action_user_list = action_user_list['user_score'].to_list()

In [ ]:
# Filter again by action
sports_user_list = df_action_sports[df_action_sports['genre'].isin(sports)]

In [ ]:
# List of user scores
sports_user_list = sports_user_list['user_score'].to_list()

In [ ]:
# Comparison of Action and Sports user ratings

#Null Hypothesis

## The mean user ratings of Action and Sports are the same


alpha = 0.05

results = st.ttest_ind(action_user_list, sports_user_list)

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis, the means are different")
else:
    print("We can't reject the null hypothesis") 

The mean action and sports user ratings are not statistically different. 

## Overall Conclusions

Overall, we can look at our data to make predictions for 2017. First, inferences would need to be made as to the region we are most interested in marketing to. Since the North American region generally leads in sales, and since the European region follows a similar trend, we can maximize our results by focusing on North America. Since the most popular platform is the PS4, we will be looking for a game on that platform. Since the most popular genre in North America is action, we will be looking for the next big action game. Furthermore, we would want a game that could be played on XONE or PC as well, to maximize sales. Since ratings and user scores do not influence sales, we will not care too much about those factors. Yet, a game in the action genre will likely be M for mature. It is highly likely that the next iteration of Call of Duty or GTA will be widely successful. Alternatively, if we want to market to the Japanese region, we will look for a role playing game on the 3DS. However, profits are likely to be higher working with the previous strategy. 